# EDA

In [ ]:
import numpy  as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import gc
import warnings
warnings.filterwarnings("ignore")

#将图表嵌入到notebook中
%matplotlib inline   

In [ ]:
import os
print(os.listdir('/kaggle/input/ashrae-energy-prediction/'))

In [ ]:
%%time
root='/kaggle/input/ashrae-energy-prediction/'
train_df=pd.read_csv(root+'train.csv')
#转换日期格式，方便以后的处理
train_df["timestamp"] = pd.to_datetime(train_df["timestamp"], format='%Y-%m-%d %H:%M:%S')
weather_train_df = pd.read_csv(root + 'weather_train.csv')
weather_test_df = pd.read_csv(root + 'weather_test.csv')

weather_train_df["timestamp"] = pd.to_datetime(weather_train_df["timestamp"], format='%Y-%m-%d %H:%M:%S')
weather_test_df["timestamp"] = pd.to_datetime(weather_test_df["timestamp"], format='%Y-%m-%d %H:%M:%S')

building_meta_df = pd.read_csv(root + 'building_metadata.csv')

test_df = pd.read_csv(root + 'test.csv')
test_df["timestamp"] = pd.to_datetime(test_df["timestamp"], format='%Y-%m-%d %H:%M:%S')

sample_submission = pd.read_csv(root + 'sample_submission.csv')

# size of data

In [ ]:
print('Size of train_df data', train_df.shape)
print('Size of weather_train_df data', weather_train_df.shape)
print('Size of weather_test_df data', weather_test_df.shape)
print('Size of building_meta_df data', building_meta_df.shape)
print('Size of test_df data', test_df.shape)
print('Size of sample_submission data', sample_submission.shape)

# Take a quick look at each data

In [ ]:
train_df.head()

In [ ]:
weather_train_df.head()

In [ ]:
weather_test_df.head()

In [ ]:
building_meta_df.head()

In [ ]:
test_df.head()

In [ ]:
sample_submission.head()

# we can learn lot from Data  download homepage

building_metadata.csv
> - site_id:0-15
> - building_id:0-1448
> - primary_use:Education Office an other(14) total 16
> - square_feet:283-875k
> - year_built:1900-2017
> - floor_count:1-26

train.csv
> - timestamp :2016.1.1-2017.1.1
> - 0: electricity, 1: chilledwater, 2: steam, 3: hotwater
> - total data is a year

test.csv
> - timestamp :2017.1.1-2019.1.1
> - total data is 2 year

In [ ]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Reduce Memory

In [ ]:
train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)

weather_train_df = reduce_mem_usage(weather_train_df)
weather_test_df = reduce_mem_usage(weather_test_df)
building_meta_df = reduce_mem_usage(building_meta_df)

# Exploratory Data Analysis

 Exploratory Data Analysis (EDA) is an open-ended process where we calculate statistics and make figures to find trends, anomalies, patterns, or relationships within the data.
 
 glimpse at meter_reading(y)

In [ ]:
plt.figure(figsize=(15,5))
train_df['meter_reading'].plot()

# Analyse train.csv

look that by ganfear:[ Missing data and zeros visualized](https://www.kaggle.com/ganfear/missing-data-and-zeros-visualized)

divide train data into building meter and time,
then seek missing data and zeros and the number of data

In [ ]:
#set_index
train=train_df.set_index(['timestamp'])

#plot missing values per building/meter
f,a=plt.subplots(1,4,figsize=(20,30))
for meter in np.arange(4):
    df=train[train.meter==meter].copy().reset_index()
    df['timestamp']=pd.to_timedelta(df.timestamp-pd.to_datetime("2016-01-01")).dt.total_seconds()/3600
    df['timestamp']=df.timestamp.astype(int)
    df.timestamp=df.timestamp-df.timestamp.min()
    missmap=np.empty((1449,df.timestamp.max()+1))
    missmap.fill(np.nan)
    for l in df.values:
        #print(l)
        if l[2]!=meter:continue
        missmap[int(l[1]),int(l[0])]=0 if l[3]==0 else 1
    a[meter].set_title(f'meter {meter:d}')
    sns.heatmap(missmap,cmap='Paired',ax=a[meter],cbar=True)
        
        

Legend:
* X axis: hours elapsed since Jan 1st 2016, for each of the 4 meter types
* Y axis: building_id
* Brown: meter reading available with non-zero value
* Light blue: meter reading available with zero value
* White: missing meter reading

we can draw some question:
* building_id 0-100 time0-2315 meter_reading is all zero they are error data,so we should clear them
* meter1 and meter3 are chillwater and hotwater so some building have Seasonal characteristics


# Examine Missing Values

checking missing data for train_df

In [ ]:
total = train_df.isnull().sum().sort_values(ascending=False)
percent=(train_df.isnull().sum()/train_df.isnull().count()*100).sort_values(ascending=False)
missing_train_data=pd.concat([total,percent],axis=1,keys=['Total','Percent'])
missing_train_data

checking missing data for weather_train_df

In [ ]:
# checking missing data
total = weather_train_df.isnull().sum().sort_values(ascending = False)
percent = (weather_train_df.isnull().sum()/weather_train_df.isnull().count()*100).sort_values(ascending = False)
missing_weather_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_weather_data.head(9)

checking missing data for weather_test_df

In [ ]:
# checking missing data
total = weather_test_df.isnull().sum().sort_values(ascending = False)
percent = (weather_test_df.isnull().sum()/weather_test_df.isnull().count()*100).sort_values(ascending = False)
missing_weather_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_weather_data.head(9)

checking missing data for building_meta_df

In [ ]:
# checking missing data
total = building_meta_df.isnull().sum().sort_values(ascending = False)
percent = (building_meta_df.isnull().sum()/building_meta_df.isnull().count()*100).sort_values(ascending = False)
missing_building_meta_df  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_building_meta_df.head(9)

In [ ]:
train_df=train_df.merge(building_meta_df,on='building_id',how='left')
test_df=test_df.merge(building_meta_df,on='building_id',how='left')

train_df=train_df.merge(weather_train_df,on=['site_id','timestamp'],how='left')
test_df=test_df.merge(weather_test_df,on=['site_id','timestamp'],how='left')


gc.collect()

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(14, 6), dpi=100)
train_df[['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes, label='By hour', alpha=0.7).set_ylabel('Meter reading', fontsize=14);
train_df[['timestamp', 'meter_reading']].set_index('timestamp').resample('D').mean()['meter_reading'].plot(ax=axes, label='By day', alpha=1).set_ylabel('Meter reading', fontsize=14);
axes.set_title('Mean Meter reading by hour and day', fontsize=16);

#显示图中的标签
axes.legend();

In [ ]:
train_df.head()

meter_reading 按照小时以及天的平均值 从3月到7月以及11月的一段时间值很大，这有些奇怪。

需要再做一下详细的分析。

site_id 有16个，可以分别看一下这16个site的meter_reading

In [ ]:
fig, axes = plt.subplots(8,2,figsize=(15, 30), dpi=100)
for i in range(train_df['site_id'].nunique()):
    train_df[train_df['site_id'] == i][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[i%8][i//8], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
    train_df[train_df['site_id'] == i][['timestamp', 'meter_reading']].set_index('timestamp').resample('D').mean()['meter_reading'].plot(ax=axes[i%8][i//8], alpha=1, label='By day', color='tab:orange').set_xlabel('');
    axes[i%8][i//8].legend();
    axes[i%8][i//8].set_title('site_id {}'.format(i), fontsize=13);
    plt.subplots_adjust(hspace=0.45)

we can see that the meter_reading of site_id 13 is too big and very like total data 

we can see it deeply

Mean meter reading by primary_use for site_id==13

In [ ]:
fig, axes = plt.subplots(8,2,figsize=(14, 30), dpi=100)
for i, use in enumerate(train_df['primary_use'].value_counts().index.to_list()):
    try:
        train_df[(train_df['site_id'] == 13) & (train_df['primary_use'] == use)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[i%8][i//8], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
        train_df[(train_df['site_id'] == 13) & (train_df['primary_use'] == use)][['timestamp', 'meter_reading']].set_index('timestamp').resample('D').mean()['meter_reading'].plot(ax=axes[i%8][i//8], alpha=1, label='By day', color='tab:orange').set_xlabel('');
        axes[i%8][i//8].legend();
    except TypeError:
        pass
    axes[i%8][i//8].set_title(use, fontsize=13);
    plt.subplots_adjust(hspace=0.45)



One more level of conclusions:

* site_id == 13 and primary_use == Education looks a lot like a general mean for meter reading. So it is really invest a lost into the whole data
* site_id == 13 and primary_use == Technology/Science also have 0's in meter readings in January. Just like site_id 0, that we found earlier.

Lets keep digging and see what meter type is responsible for such weird look of the meter reading for site_id 13 and primary_use Education.

Mean meter reading by meter type for primary_use==Education and site_id==13

In [ ]:
fig, axes = plt.subplots(3,1,figsize=(14, 18), dpi=100)
for i in train_df[(train_df['site_id'] == 13) & (train_df['primary_use'] == 'Education')]['meter'].value_counts(dropna=False).index.to_list():
    train_df[(train_df['site_id'] == 13) & (train_df['primary_use'] == 'Education') & (train_df['meter'] == i)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[i], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
    train_df[(train_df['site_id'] == 13) & (train_df['primary_use'] == 'Education') & (train_df['meter'] == i)][['timestamp', 'meter_reading']].set_index('timestamp').resample('D').mean()['meter_reading'].plot(ax=axes[i], alpha=1, label='By day', color='tab:orange').set_xlabel('');
    axes[i].legend();
    axes[i].set_title('Meter: ' + str(i), fontsize=13);

Fine. What we have right now is that site_id 13 with primary_use Education and meter type 2 is responsible for this mess.
But we need to go deeper. There are 17 buildings that fall under these criteria. No problem. Another plot.

In [ ]:
len(train_df[(train_df['primary_use']=='Education')&(train_df['site_id']==13)&(train_df['meter']==2)]['building_id'].value_counts().index.to_list())

In [ ]:
fig, axes = plt.subplots(9,2,figsize=(14, 36), dpi=100)
for i, building in enumerate(train_df[(train_df['site_id'] == 13) & (train_df['primary_use'] == 'Education') & (train_df['meter'] == 2)]['building_id'].value_counts(dropna=False).index.to_list()):
    train_df[(train_df['site_id'] == 13) & (train_df['primary_use'] == 'Education') & (train_df['meter'] == 2) & (train_df['building_id'] == building)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[i%9][i//9], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
    train_df[(train_df['site_id'] == 13) & (train_df['primary_use'] == 'Education') & (train_df['meter'] == 2) & (train_df['building_id'] == building)][['timestamp', 'meter_reading']].set_index('timestamp').resample('D').mean()['meter_reading'].plot(ax=axes[i%9][i//9], alpha=1, label='By day', color='tab:orange').set_xlabel('');
    axes[i%9][i//9].legend();
    axes[i%9][i//9].set_title('building_id: ' + str(building), fontsize=13);
    plt.subplots_adjust(hspace=0.45)

After going through all the buildings, we can find the problem ,It is building 1099

In [ ]:
fig, axes = plt.subplots(3,1,figsize=(14, 20), dpi=100)

train_df[(train_df['meter'] == 2) & (train_df['building_id'] == 1099)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[0], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
train_df[(train_df['meter'] == 2) & (train_df['building_id'] == 1099)][['timestamp', 'meter_reading']].set_index('timestamp').resample('D').mean()['meter_reading'].plot(ax=axes[0], alpha=1, label='By day', color='tab:orange').set_xlabel('');

train_df[['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[1], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
train_df[['timestamp', 'meter_reading']].set_index('timestamp').resample('D').mean()['meter_reading'].plot(ax=axes[1], alpha=1, label='By day', color='tab:orange').set_xlabel('');

train_df[~((train_df['meter'] == 2) & (train_df['building_id'] == 1099))][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[2], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
train_df[~((train_df['meter'] == 2) & (train_df['building_id'] == 1099))][['timestamp', 'meter_reading']].set_index('timestamp').resample('D').mean()['meter_reading'].plot(ax=axes[2], alpha=1, label='By day', color='tab:orange').set_xlabel('');

axes[0].set_title('building_id==1099 and meter==2', fontsize=13);
axes[1].set_title('Full dataset', fontsize=13);
axes[2].set_title('building_id 1099 excluded', fontsize=13);
plt.subplots_adjust(hspace=0.45)

 so we should clean data of building_id ==1099 and meter ==2 

### Correlations
One way to try and understand the data is by looking for correlations between the features and the target. We can calculate the Pearson correlation coefficient between every variable and the target using the `.corr` dataframe method.

* .00-.19 “very weak”
*  .20-.39 “weak”
*  .40-.59 “moderate”
*  .60-.79 “strong”
* .80-1.0 “very strong”

In [ ]:
# Find correlations with the target and sort
correlations = train_df.corr()['meter_reading'].sort_values()

# Display correlations
print('Most Positive Correlations:\n', correlations.tail(7))
print('\nMost Negative Correlations:\n', correlations.head(4))

corrs = train_df.corr()
corrs

<font color=#000000 size=6 face="宋体"> plot dist curves for train and  test weather data for the given column name </font>

In [ ]:
def plot_dist_col(column):
    fig,ax=plt.subplots(figsize=(10,10))
    sns.distplot(weather_train_df[column].dropna(),color='green',ax=ax).set_title(column,fontsize=16)
    sns.distplot(weather_test_df[column].dropna(),color='purple',ax=ax).set_title(column,fontsize=16)
    plt.xlabel(column,fontsize=15)
    plt.legend(['train','test'])
    plt.show()

In [ ]:
plot_dist_col('air_temperature')

In [ ]:
plot_dist_col('cloud_coverage')

In [ ]:
weather_train_df['cloud_coverage'].unique()

In [ ]:
plot_dist_col('dew_temperature')

In [ ]:
plot_dist_col('precip_depth_1_hr')

In [ ]:
weather_train_df['precip_depth_1_hr'].unique()

In [ ]:
plot_dist_col('sea_level_pressure')

In [ ]:
plot_dist_col('wind_direction')

In [ ]:
plot_dist_col('wind_speed')

In [ ]:
#风速小于0，这里不确定是数据有问题还是因为逆风
len(weather_train_df[weather_train_df['precip_depth_1_hr']<0])

<font color=#000000 size=6 face="宋体"> plot dist curves for building data for the given column name </font>

In [ ]:
def plot_dist_col(column):
    fig,ax=plt.subplots(figsize=(10,10))
    sns.distplot(building_meta_df[column].dropna(),color='green',ax=ax).set_title(column,fontsize=16)
       
    plt.xlabel(column,fontsize=15)
    plt.legend(['building'])
    plt.show()

In [ ]:
plot_dist_col('floor_count')

In [ ]:
plot_dist_col('year_built')

In [ ]:
building_meta_df['year_built'].unique()

In [ ]:
plot_dist_col('square_feet')

<font color='red' size=4 face="微软雅黑">square_feet的值比较大，应该做平滑处理 </font>

<font color=#000000 size=6 face="微软雅黑">simple time series analysis </font>

In [ ]:
train_df_sub_1099=train_df[~((train_df['meter'] == 2) & (train_df['building_id'] == 1099))]
ts=train_df_sub_1099.groupby(["timestamp"])["meter_reading"].mean()
ts.astype('float')
plt.figure(figsize=(16,8))
plt.title('average meter_reading with time')
plt.xlabel('timestamp')
plt.ylabel('meter_reading')
plt.plot(ts)

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(ts.rolling(window=12).mean(),label='Rolling Mean')
plt.plot(ts.rolling(window=12).std(),label='Rolling sd')
plt.legend()

标准差是有规律的，可能周期是一星期？

<font  size=4 face="微软雅黑">查看每个site_id 各有多少类型的建筑 </font>

In [ ]:
building_meta_df.groupby('site_id').primary_use.agg(lambda x:x.value_counts().to_dict()).to_dict()

In [ ]:
building_meta_df.groupby('site_id').building_id.agg(lambda x:x.value_counts().to_dict()).to_dict()[0]

<font  size=4 face="微软雅黑">查看每个site_id 的24小时的平均温度数据 </font>

In [ ]:
fig, axes = plt.subplots(8,2,figsize=(14, 20), dpi=100)
#weather_train_df[weather_train_df['site_id']==0].plot()

import datetime
# plt.figure(figsize=(16,6))

# plt.xlabel('timestamp')
# plt.ylabel('air_temperature')

#plt.plot(weather_train_df[])
#plt.legend()
weather_train_df['hour']=weather_train_df.timestamp.dt.hour
weather_train_df_mean_by_hour=weather_train_df.groupby(['site_id','hour']).mean()
# #train_df[train_df['site_id'] == i][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[i%8][i//8], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
for i in range(16):
    weather_train_df_mean_by_hour[i*24:(i+1)*24]['air_temperature'].plot(ax=axes[i%8][i//8])
    axes[i%8][i//8].legend();
    axes[i%8][i//8].set_title('site_id {} max temperature hour is {}'.format(i,np.argmax(weather_train_df_mean_by_hour[i*24:(i+1)*24]['air_temperature'])[1]), fontsize=13);
plt.subplots_adjust(hspace=0.8)

<font  size=3 face="微软雅黑">  &ensp; &ensp; 从上图我们可以发现一些问题，不同的site_id 的最高温度在不同的时间，比如site_id=13地区的最高温度在22点或23点左右，这是有问题的。这说明weather_data里面的timestamp是有问题的。我们需要校准时间，这样和traain_df表连接后才是有意义的。</font>

<font  size=3 face="微软雅黑">  &ensp; &ensp; 温度最高的时间应该是在14：00从上图我们 每个site_id的最高温度时间中，我们可以计算出时间差。</font> <br/><br/>&ensp; &ensp; <font  size=3 face="微软雅黑" color="blue">我们将会使用这个差来重新调整weather数据中的timestamp    site_GMT_offsets = [-5, 0, -9, -6, -8, 0, -6, -6, -5, -7, -8, -6, 0, -7, -6, -6]</font>

<font  size=6 face="微软雅黑">缺失值填补探索
</font>

<font  size=3 face="微软雅黑">  &ensp; &ensp; 从建模后的特征重要性以及尝试各种特征后得出 air_temperature以及dew_temperature
是和能源使用情况最相关，最重要的特征。训练集中的 air_temperature和dew_temperature分别有 4%和8%的缺失，测试集中分别有 4%和11%的缺失。
</font>

<font  size=5 face="微软雅黑">  &ensp; &ensp; air_temperature</font>

#load training data 2016
weather=pd.read_csv(root + 'weather_train.csv',parse_dates=['timestamp'])
weather.head()#pivot to plot

In [ ]:
#load training data 2016
weather_train=pd.read_csv(root+'weather_train.csv', parse_dates=['timestamp'])
weather_train.head()

In [ ]:
#pivot to plot
wmatrix_train=weather_train.pivot(index='timestamp',columns='site_id',values='air_temperature')
wmatrix_train.head()

In [ ]:
#寻找训练集中有最多缺失值的site_id
site_id=wmatrix_train.count().idxmin()

#挑选出一段时间进行观测
start_date,end_date=datetime.date(2016,1,1),datetime.date(2016,1,9)

#初始化绘图
f,ax=plt.subplots(figsize=(18,6))

#load test data 2017-2018
weather_test=pd.read_csv(root+'weather_test.csv', parse_dates=['timestamp'])

#shift 2017 to 2016
weather_test.timestamp=weather_test.timestamp-datetime.timedelta(365)
wtmatrix=weather_test.pivot(index='timestamp',columns='site_id',values='air_temperature')
wtmatrix.loc[start_date:end_date,site_id].plot(ax=ax,label=f'2017.1.1-2017.1.9 site:{site_id}',alpha=0.5)

#shift 2018 to 2016
weather_test.timestamp=weather_test.timestamp-datetime.timedelta(365)
wtmatrix=weather_test.pivot(index='timestamp',columns='site_id',values='air_temperature')
wtmatrix.loc[start_date:end_date,site_id].plot(ax=ax,label=f'2018.1.1.-2018.1.9 site:{site_id}',alpha=0.5)


def fill_with_polynomial(wmatrix):
    return wmatrix.fillna(wmatrix.interpolate(method='polynomial',order=3))

def fill_with_lin(wmatrix):
    return wmatrix.fillna(wmatrix.interpolate(method='linear'))

def fill_with_mix(wmatrix):
    wmatrix=(wmatrix.fillna(wmatrix.interpolate(method='linear',limit_direction='both'))+ wmatrix.fillna(wmatrix.interpolate(method='polynomial', order=3, limit_direction='both')))*0.5  
    
    return wmatrix        # fill with second item
    
fill_with_lin(wmatrix_train).loc[start_date:end_date,site_id].plot(ax=ax,label=f'linear Jan 2016 site:{site_id}',alpha=0.5) 
fill_with_polynomial(wmatrix_train).loc[start_date:end_date,site_id].plot(ax=ax,label=f'polynomial Jan 2016 site:{site_id}',alpha=0.5)    
fill_with_mix(wmatrix_train).loc[start_date:end_date,site_id].plot(ax=ax,label=f'mix Jan 2016 site:{site_id}',alpha=0.5)    
wmatrix_train.loc[start_date:end_date, site_id].plot(ax=ax, label=f'Jan 2016 site:{site_id}')
plt.legend()

<font  size=3 face="微软雅黑" color="blue">  &ensp; &ensp; 看起来 是线性和多项式1:1混合修补缺失值比较平滑，但在建模的时候3种方法都尝试后发现，线性补值最好
</font>

### same for dew_temperature

In [ ]:
# pivot to plot
col = 'dew_temperature'
wmatrix = weather_train.pivot(index='timestamp', columns='site_id', values=col)
# site with largest amount of missing data points
site_id = wmatrix.count().idxmin()
# plot perid
start_date, end_date = datetime.date(2016, 1, 1), datetime.date(2016, 1, 12)
f,ax = plt.subplots(figsize=(18,6))

_ = fill_with_lin(wmatrix).loc[start_date:end_date, site_id].plot(ax=ax, label=f'linear Jan 2016 site:{site_id}', alpha=0.5)
_ = fill_with_polynomial(wmatrix).loc[start_date:end_date, site_id].plot(ax=ax, label=f'cubic Jan 2016 site:{site_id}', alpha=0.5)
_ = fill_with_mix(wmatrix).loc[start_date:end_date, site_id].plot(ax=ax, label=f'mix Jan 2016 site:{site_id}', alpha=0.5)
_ = wmatrix.loc[start_date:end_date, site_id].plot(ax=ax, label=f'Jan 2016 site:{site_id}')

_ = plt.legend()

<font  size=3 face="微软雅黑" color="blue">  &ensp; &ensp; 同样看起来 是线性和多项式1:1混合修补缺失值比较平滑，但在建模的时候3种方法都尝试后发现，线性补值最好
</font>